In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

Importing all the required libraries for reading the HTML + Cleaning the HTML

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

Checking if the page has been read correctly

In [4]:
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [5]:
table = soup.find('table')

In [6]:
rows = table.find_all('td')

In [8]:
str_cells = str(rows)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()

Cleaning the data table and converting it into a DataFrame

In [9]:
list = cleantext.split("\n")
df = pd.DataFrame(list)
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3
0,[M1A,Not assigned,Not assigned,None
1,,M2A,Not assigned,Not assigned
2,,M3A,North York,Parkwoods
3,,M4A,North York,Victoria Village
4,,M5A,Downtown Toronto,Harbourfront
5,,M5A,Downtown Toronto,Regent Park
6,,M6A,North York,Lawrence Heights
7,,M6A,North York,Lawrence Manor
8,,M7A,Queen's Park,Not assigned
9,,M8A,Not assigned,Not assigned


In [10]:
#Naming the columns as required
df1 = df1.drop(columns=0)
df1.iloc[0,0] = 'M1A'
df1.columns=["PostalCode","Borough","Neighbourhood"]
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
#Deleting Rows where the Borough is not assigned
df2=df1
df2['Borough'] = df2['Borough'].str.strip()
df3 = df2
df3.drop(df3[df3.Borough == 'Not assigned'].index, inplace=True)

In [12]:
df3.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
#for Neighrbourhoods which are not assigned, assigning the value of the Borough
df3['Neighbourhood'] = df3['Neighbourhood'].str.strip()
df3.loc[df3['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df3['Borough']

In [14]:
df3.drop(df3.index[[211]], inplace=True)

Final Shape of the DataFrame after cleaning is (211, 3)

In [15]:
#Part 1 - Final Shape of the Required Data Frame
df3.shape

(211, 3)

In [16]:
df4 = df3.reset_index(drop=True)

In [17]:
df4.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#Part 2 - Merging with the Geo File

In [18]:
latlon = pd.read_csv('http://cocl.us/Geospatial_data')

In [19]:
#removing spaces
latlon['Postal Code'] = latlon['Postal Code'].str.lstrip()
df4['PostalCode'] = df4['PostalCode'].str.strip()

In [20]:
#creating the merged dataframe
merged_left = pd.merge(left=df4,right=latlon, left_on='PostalCode', right_on='Postal Code', how='left', indicator=True)

In [21]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,_merge
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,both
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,both
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,both
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636,both
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763,both


In [22]:
#Removing extra columns
merged_left.drop(["Postal Code", "_merge"], axis=1, inplace=True)

In [24]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Above is the required dataframe with the latitude and longitude added

# Part 3 - Generating Maps for the created dataframe

In [27]:
#Importing Required Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-

In [28]:
merged_left.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [30]:
#select Neighbourhoods from Toronto only
toronto_data = merged_left[merged_left['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [31]:
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Toronto Latitude and Longitude

In [32]:
latitude = 43.6532
longitude = -79.3832

In [51]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    

map_toronto    

### I decided to cluster the neighbourhood based on their locations - thus cluster nearby neighbourhoods together

In [38]:
#creating 5 clusters
kclusters = 5
toronto_clusters = toronto_data.drop(["PostalCode","Borough", "Neighbourhood"], axis=1)
toronto_clusters.head()

,Latitude,Longitude
0,43.654260,-79.360636
1,43.654260,-79.360636
2,43.657162,-79.378937
3,43.657162,-79.378937
4,43.651494,-79.375418


In [39]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clusters)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 4, 3, 3, 0, 3], dtype=int32)

In [40]:
toronto_clusters.head()

,Latitude,Longitude
0,43.654260,-79.360636
1,43.654260,-79.360636
2,43.657162,-79.378937
3,43.657162,-79.378937
4,43.651494,-79.375418


In [41]:
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [42]:
toronto_data['Cluster Labels'] = kmeans.labels_

In [44]:
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3
5,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,3


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Above is the final Toronto map clustered based on location